## Paper Plot

In [ ]:
import pandas as pd

df = pd.read_csv("/root/cache/repsim_subgroups.csv", index_col=0)
df.loc[df["k"].isna(), "k"] = -1
df.loc[df["perc_k"].isna(), "perc_k"] = -1

df.loc[df.metric == "proc", "score"] = (2 - df.loc[df.metric == "proc", "score"]) / 2
df.loc[df.metric == "rtd", "score"] = -1 *  df.loc[df.metric == "rtd", "score"]

df.head()

In [35]:
# df = df.loc[~((~df.metric.isin(["cka", "proc"])) & (df.eps == 3))]#.groupby(["k", "perc_k", "metric", "pred_type", "eps"], dropna=False)["score"].describe()

In [ ]:
df.groupby(["k", "perc_k", "metric", "pred_type", "eps"], dropna=False)["score"].describe()

In [ ]:
df["metric_id"] = df["metric"] + "_" + df["k"].astype(str)+ "_" + df["perc_k"].astype(str)
df

In [ ]:
import seaborn as sns

plotdata = df.loc[(df.model1 != df.model2)]
sns.catplot(data=plotdata, y="score", hue="pred_type", x="eps", col="metric_id", col_wrap=5, sharey=False, kind="boxen")

In [ ]:
from matplotlib.colors import Normalize, LinearSegmentedColormap
import matplotlib.pyplot as plt

sns.set_style("darkgrid")

metrics = ["cka_-1.0_-1.0", "proc_-1.0_-1.0", "rtd_-1.0_-1.0", "jacc_10.0_-1.0"]
# plotdata = df.loc[(df.model1 != df.model2) & (df.metric_id.isin(metrics))]
# sns.catplot(data=plotdata, y="score", hue="pred_type", x="eps", col="metric_id", sharey=False, kind="boxen")

n_panels = 4
models = ["resnet18", "resnet50", "wide_resnet50_2", "wide_resnet50_4", "densenet161", "vgg16", "resnext50_32x4d"]

fig, axes = plt.subplots(1, n_panels, figsize=(n_panels*3*1.61, 3))
ax_idx = 0
xlabel = r"Robustness ($\epsilon$)"

plotdata = df.loc[(df.metric_id == "cka_-1.0_-1.0") & (df.model1 != df.model2) & (df.model1.isin(models)) & (df.model2.isin(models))]
ax = axes[ax_idx]
sns.boxplot(plotdata, x="eps", y="score", ax=ax, hue="pred_type", palette="Set2", legend=False)
# sns.stripplot(data=plotdata, x="eps", y="score", ax=ax, hue="pred_type", dodge=True, alpha=0.8)
ax.set_title("CKA")
ax.set_ylabel("Regular Representation\nSimilarity")
ax.set_xlabel(xlabel)
ax_idx += 1

plotdata = df.loc[(df.metric_id == "proc_-1.0_-1.0") & (df.model1 != df.model2) & (df.model1.isin(models)) & (df.model2.isin(models))]
ax = axes[ax_idx]
sns.boxplot(plotdata, x="eps", y="score", ax=ax, hue="pred_type", palette="Set2", legend=False)
# sns.stripplot(data=plotdata, x="eps", y="score", ax=ax, hue="pred_type", dodge=True, alpha=0.8)
ax.set_title("ProcrustesSim")
ax.set_ylabel("")
ax.set_xlabel(xlabel)
ax_idx += 1

plotdata = df.loc[(df.metric_id == "rtd_-1.0_-1.0") & (df.model1 != df.model2) & (df.model1.isin(models)) & (df.model2.isin(models))]
ax = axes[ax_idx]
sns.boxplot(plotdata, x="eps", y="score", ax=ax, hue="pred_type", palette="Set2", legend=False)
# sns.stripplot(data=plotdata, x="eps", y="score", ax=ax, hue="pred_type", dodge=True, alpha=0.8)
ax.set_title("neg. RTD")
ax.set_ylabel("")
ax.set_xlabel(xlabel)
ax_idx += 1

plotdata = df.loc[(df.metric_id == "jacc_10.0_-1.0") & (df.model1 != df.model2) & (df.model1.isin(models)) & (df.model2.isin(models))]
ax = axes[ax_idx]
sns.boxplot(plotdata, x="eps", y="score", ax=ax, hue="pred_type", palette="Set2", legend=True)
# sns.stripplot(data=plotdata, x="eps", y="score", ax=ax, hue="pred_type", dodge=True, alpha=0.8)
ax.set_title("Jaccard")
ax.set_ylabel("")
ax.set_xlabel(xlabel)
sns.move_legend(ax, "center", bbox_to_anchor=(1.2,0.5), title="Predictions")

ax_idx += 1
fig.savefig("../figs/agree_vs_disagree_in1k.pdf", bbox_inches="tight")

In [ ]:
import scipy.stats

data_agg = df.loc[(df.metric=="rtd") & (df.model1 != df.model2) & (df.eps==3.0) & (df.pred_type=="agree"), "score"].values
data_disag = df.loc[(df.metric=="rtd") & (df.model1 != df.model2) & (df.eps==3.0) & (df.pred_type=="disagree"), "score"].values

def mean_diff(x, y, axis):
    return x.mean(axis=axis) - y.mean(axis=axis)

scipy.stats.permutation_test([data_agg, data_disag], mean_diff, permutation_type="samples", vectorized=True)